# Développer une application web dynamique cartographique avec Panel (*ou comment construire un jeu interactif ?*)

> ***[Lien direct vers le jeu](https://thomleysens.github.io/Tutoriels_AME/game_panel.html)***

> ***AVERTISSEMENT: il existe de nombreuses façons d'améliorer le code de notre application. Néanmoins, nous avons souhaité essayer de le rendre le plus simple et le plus accessible possible tout en restant "pythonique".***


| Sujet | Auteur | Dernière Màj |
|:---:|:------:|:------------:|
| Application webmapping | Thomas LEYSENS (*IR AME*) | *05 décembre 2023* |


## Enjeux & Propositions
| Enjeu | Proposition |
|:------|:------------|
| Global | Développer une application de webmapping (*découvrir les possibilités de la librairie Python Panel*) |
| Classe et fonctions Python | Explications et exemples |
| Structurer une application | Widgets, figures et callbacks |
| Utiliser des paramètres externes | Charger et lire un fichier YAML |
| Utiliser des données externes | Charger et lire un fichier CSV avec Pandas et Geopandas |
| Contraintes d'une application en ligne | Différences entre local et en ligne |

## Préparation
### Données

Dans notre cas, nous souhaitons construire un fichier de questions/réponses au format CSV (*comma-separated values*) afin qu'il soit chargé dans notre application:
* c'est un jeu cartographique, nous allons donc récupérer des données sur [OpenStreetMap](https://www.openstreetmap.org/)
* nous souhaitons agrémenter les réponses d'un lien Wikipédia (*quand il est disponible*), un prétraitement est donc nécessaire (*afin de ne pas relancer les mêmes requêtes web lors de chaque lancement de notre application*). 

#### OpenStreetMap

> ***AVERTISSEMENT: Veillez à bien respecter les conditions d'utilisation des données open source d'OpenStreetMap, cf. [License Open Data Commons Open Database License (ODbL)](https://opendatacommons.org/licenses/odbl)***

Allons chercher quelques lieux symboliques de Le Croisic sur [OpenStreetMap](https://www.openstreetmap.org/), voici comment faire pour l'exemple en question (*Pierre Longue*):

![OSM query](../img/game/OSM.png)

#### CSV

Petit à petit, nous pouvons construire notre [fichier CSV](https://raw.githubusercontent.com/thomleysens/Tutoriels_AME/main/tutoriels/game_QA.csv) de questions/réponses avec tous les détails souhaités, dont voici un extrait ci-dessous:

| question | answer | hint | x | y | wikidata |
|:---------|:-------|:-----|--:|--:|:---------|
| My eye, when opened, can save the ships. Who am I ? | Phare du Tréhic | Search in the North | -2.5237 | 47.3082 | Q20971192 |
| If you break your leg, where do you go ? | Ancien hôpital du Croisic | Search for the red circle with a cross | -2.51493 | 47.2918 | Q21418896 |
| I've got wheat and I want flour, where do I have to go ? | Moulin de Beauvran | Search near south of Laboratoire de biologie marine | -2.51796 | 47.2855 | Q21152416 |
| I'm BotMan, where do I park my Botmobile ? | Manoir de Kervaudu | It is a manor surrounded by green areas | -2.5264  | 47.2964 | Q3286209 |

On peut constater qu'il nous manque les URL des pages Wikipedia (*elles sont parfois présentes dans les données OSM mais nous préférons utiliser le Wikidata car il est plus souvent renseigné*). Il est possible de les retrouver via l'identifiant [Wikidata](https://fr.wikipedia.org/wiki/Wikidata). Plutôt que de faire des recherches manuelles, nous allons créer une méthode Python pour récupérer automatiquement l'information et compléter le fichier CSV (*le fichier Python se trouve [ici](https://github.com/thomleysens/Tutoriels_AME/blob/main/tutoriels/utils.py) et nous allons le détailler et l'expliquer*). 

Tout d'abord, voici le code brut:

```python
"""
@author: thomleysens
"""

import requests
import pandas as pd


def get_wiki_url(
    wikidata_id,
    base_url="https://www.wikidata.org/wiki/Special:EntityData/{}.json",
    wiki="frwiki"
):
    """
    Get wikipedia url from wikidata ID

    Parameters
    ----------
    wikidata_id (str): Wikidata ID
    base_url (str): Url to request Entity and get JSON response
                    Default: "https://www.wikidata.org/wiki/Special:EntityData/{}.json
    wiki (str): Wiki reference
                Default: "frwiki"

    Returns
    -------
    url (str):
        - Wikipedia URL if wikidata_id is not NA
        - None if wikidata_id is NA
    """
    if pd.isna(wikidata_id) is False:
        url = requests.get(
            base_url.format(wikidata_id)
        ).json()["entities"][wikidata_id]["sitelinks"][wiki]["url"]
    else:
        url = None

    return url
```

Détaillons et explicitons le:
* Tout d'abord, un commentaire indiquant l'auteur du code (*cette partie du code ne sera pas exécutée étant considérée comme un commentaire*):

```python
"""
@author: thomleysens
"""
```
  
* Il est nécessaire, ensuite, d'importer les librairies/modules Python qui seront utilisés dans notre code (*[requests](https://requests.readthedocs.io/en/latest/) et [pandas](https://pandas.pydata.org/)*):

```python
import requests
import pandas as pd
```
  
* Vient ensuite la fonction/méthode Python qui nous intéresse. Il convient de respecter [certaines conventions](https://peps.python.org/pep-0008/#function-and-variable-names) pour la nommer. Le nom de la fonction vient après le mot clef ```def``` qui définit une fonction. Nous indiquons entre parenthèses les arguments qui peuvent être [des ***positional arguments*** ou des ***keywords arguments***](https://stackoverflow.com/a/9450673).

```python
def get_wiki_url(
wikidata_id,
base_url="https://www.wikidata.org/wiki/Special:EntityData/{}.json",
wiki="frwiki"
):
```
  
* Il convient de bien renseigner la fonction (*afin qu'elle puisse êre comprise/modifiée/corrigée par d'autres*). Pour ce faire, nous utilisons des [docstrings](https://peps.python.org/pep-0257/):

```python
"""
Get wikipedia url from wikidata ID

Parameters
----------
wikidata_id (str): Wikidata ID
base_url (str): Url to request Entity and get JSON response
              Default: "https://www.wikidata.org/wiki/Special:EntityData/{}.json
wiki (str): Wiki reference
          Default: "frwiki"

Returns
-------
url (str):
- Wikipedia URL if wikidata_id is not NA
- None if wikidata_id is NA
"""
```
  
* Enfin vient le code qui sera exécuté. Tout d'abord, on vérifie que ```wikidata_id``` n'est pas ```NaN``` (*valeur nulle*) avant de continuer. Cela est nécessaire car certaines lignes de notre CSV contiennent des valeurs nulles dans la colonne ```wikidata```.

```python
if pd.isna(wikidata_id) is False:
```
    
* Si ```wikidata_id``` n'est pas ```NaN```:
  * nous formatons ```base_url``` avec la variable ```wikidata_url```:
    
    ```python
    base_url.format(wikidata_id)
    ```
  
  * nous utilisons la fonction ```get()``` de la librairie ```requests``` pour lancer une requête web avec l'url formatée et nous récupérons une valeur précise dans le [JSON](https://www.json.org/json-en.html) renvoyée par notre requête (*si nous étions meilleurs élèves, il faudrait vérifier que la requête s'est bien déroulée*): la valeur ```"url"```. Imaginons que ```wikidata_id``` ait la valeur ```Q20971192``` et que ```wiki``` ait la valeur par défaut, la valeur de l'élément ```"url"``` se trouverait au bout du "chemin" ```["entities"]["Q20971192"]["sitelinks"]["frwiki"]["url"]```:

    ```python
    url = requests.get(
        base_url.format(wikidata_id)
    ).json()["entities"][wikidata_id]["sitelinks"][wiki]["url"]
    ```

* Si ```wikidata_id``` est ```NaN```, la fonction attribue la valeur ```None``` à la variable ```url```:

```python
else:
  url = None
```
  
* La fonction renvoie enfin la variable ```url```:

```python
return url
```

Nous pouvons donc (*cf code ci-dessous*):
* lire et charger le fichier CSV ```game_QA.csv```
* importer la fonction précédente afin de l'appliquer à la colonne ```url```
* enregistrer le fichier CSV modifié

In [ ]:
import pandas as pd

from utils import get_wiki_url

csv_file = "game_QA.csv"
df = pd.read_csv(
    csv_file,
    sep=";",
    encoding="utf-8"
)
df["url"] = df["wikidata"].map(
    get_wiki_url
)
df.to_csv(
    csv_file,
    sep=";",
    encoding="utf-8",
    index=False
)

## Application Panel

> *Nous présentons ici les fonctionnalités offertes par la [librairie Python Panel](https://panel.holoviz.org/) pour la création d'une application de webmapping interactive et dynamique. Il s'agit de mettre en lumière quelques possibilités et de donner un exemple de code.*

> *Le Jupyter Notebook contenant l'application se trouve [ici](https://github.com/thomleysens/Tutoriels_AME/blob/main/tutoriels/game_panel.ipynb)*

### Enjeux

Nous avons souhaité développé une application - que nous espérons ludique - afin d'illustrer quelques possibilités pour une application interactive et dynamique avec un certain degré de complexité. Les enjeux sont les suivants:

* Exposer les possibilités d'intégrer une carte interactive
* Intégrer un exemple de chat
* Créer quelques widgets et des callbacks liés
* Montrer des exemples d'interactions
* Charger des fichiers (*paramètres et données*)
* Exposer quelques possibilités de données/outils à disposition pour la cartographie

### Explications techniques
Tout d'abord, comme pour le module Python contenant notre fonction précédente, nous importons les librairies Python que nous utiliserons dans notre classe et fonctions Python:

```python
import geopandas as gpd
import panel as pn
import numpy as np
import time
import xyzservices.providers as xyz
from random import sample
from bokeh.palettes import Greens5 as palette
from bokeh.models import (
    ColumnDataSource,
    PointDrawTool, 
    Button,
    HoverTool,
    Range1d
)
from bokeh.plotting import figure
from shapely.geometry import Point

from utils import load_params
```

Intéressons nous à la dernière ligne:

```python
from utils import load_params
```

Elle est séparée des autres puisque nous importons une fonction que nous avons créée - *et non une librairie Python installée dans notre environnement* - depuis un module que nous avons développé. 

Il est nécessaire, pour que l'application fonctionne correctement depuis le notebook, d'indiquer à Panel de charger [la ou les extensions(s) nécessaire(s)](https://panel.holoviz.org/api/cheatsheet.html#pn-extension). Les paramètres entre parenthèses peuvent, bien entendu, varier en fonction des besoins. 
```python
pn.extension(notifications=True)
```

L'application repose sur quelques paramètres que nous avons choisi de charger depuis un fichier de paramètres [YAML](https://fr.wikipedia.org/wiki/YAML) (*dont voici un exemple ci-dessous*) afin de ne pas surcharger notre application. 

Le [fichier YAML](https://github.com/thomleysens/Tutoriels_AME/blob/main/tutoriels/game_params.yml) se présente ainsi:

```yaml
ranges:
    x: [-284221, -277648]
    y: [5987515, 5992714]
buffer_value: 200
csv:
    file: "game_QA.csv"
    sep: ";" 
nb_questions: 5
```

Détaillons ces paramètres:
* ```ranges```: nous indiquons des intervalles pour l'axe des x et des y pour la figure comprenant la carte (*les valeurs sont exprimées en [EPSG](https://fr.wikipedia.org/wiki/Syst%C3%A8me_de_coordonn%C3%A9es_(cartographie)) 3857*). Ces valeurs ont été obtenues via l'outil [Tile calculator de Geofabrik](https://tools.geofabrik.de/calc/).
* ```buffer_value```: Valeur (*en mètres*) utilisée pour créer un buffer (*ou zone tampon*) lors de la vérification d'un point tracé par l'utilisateur.
* ```csv```: Indication du chemin du fichier CSV via ```file``` et du séparateur avec ```sep```.
* ```nb_questions```:  Nombre maximal de questions par session de jeu

Ceux-ci doivent être *lus* et *compris* par notre code, il faut donc que Python puisse lire notre fichier de paramètres, c'est l'objet des lignes suivantes dans notre module:

> ***Avertissement: il existe une légère différence entre la version locale et la version en ligne de notre application - pour des raisons de sécurité - puisqu'il n'est pas possible de charger un fichier "local" depuis une application. Pour la version en ligne, nous chargeons donc un fichier de paramètres hébergé en ligne sur notre dépôt GitHub qui comporte un lien vers un fichier CSV également hébergé sur notre dépôt.***

* version locale:

```python 
filepath = "./game_params.yml"
PARAMS = load_params(filepath)
```

* version en ligne:

```python
filepath = "https://raw.githubusercontent.com/thomleysens/Tutoriels_AME/main/tutoriels/game_params_remote.yml"
PARAMS = load_params(filepath)
```

```load_params``` est importée depuis ```utils.py```. Elle permet de gérer 2 cas pour charger un fichier de paramètres: depuis un fichier local ou depuis une URL.

```python
def load_params(filepath):
    """
    Load parameters from local or remote
    YAML file

    Parameters
    ----------
    filepath (str): URL or path to local file
                    File (local or remote) needs 
                    to be structured like this:
                    ranges:
                        x: [-284221, -277648]
                        y: [5987515, 5992714]
                    buffer_value: 200
                    csv:
                        file: "game_QA.csv"
                        sep: ";" 
                    nb_questions: 5
    Return
    ------
    params (namedtuple)
    """
    Params = namedtuple(
        "Params",
        [
            "ranges",
            "buffer_value",
            "nb_q",
            "csv_sep",
            "csv_file"
        ]
    )
    #Cheap way to check if url
    #(better methods exist but we keep it simple)
    if filepath.lower().startswith(
        ("http://", "https://")
    ):
        content = yaml.safe_load(
            requests.get(filepath).content
        )
    else:
        with open(filepath, "r") as file:
            content = yaml.load(
                file,
                Loader=yaml.FullLoader,
            )

    params = Params(
        content["ranges"],
        content["buffer_value"],
        content["nb_questions"],
        content["csv"]["sep"],
        content["csv"]["file"]
    )

    return params
```

Cette fonction permet de vérifier (*d'une manière un peu pauvre mais nous souhaitons garder une certaine simplicité*) si le chemin du fichier est une URL ou, à défaut, un chemin vers un fichier local:
* si c'est une URL le code lance une requête web et lit le résultat de la requête pour charger le YAML
* si c'est un chemin vers un fichier local, il charge le fichier.

La fonction renvoie un [namedtuple](https://docs.python.org/3/library/collections.html#collections.namedtuple) ```params``` qui est un objet Python (*similaire à une classe*) comprenant les éléments suivants présents dans le YAML:
* ```ranges``` => ranges x et y 
* ```buffer_value``` => valeur du buffer en mètres
* ```nb_q``` => nombre maximum de questions par session
* ```csv_sep``` => délimiteur CSV
* ```csv_file``` => chemin/URL vers le fichier CSV

Elle permet de rendre le code plus *propre* et lisible sans le complexifier à outrance. 

```cmd
panel convert game_panel.ipynb --to pyodide-worker --out pyodide
```

```cmd
python3 -m http.server
```

http://localhost:8000/pyodide/game_panel.html

## Licenses librairies Python / Données / API

### Données & API

| API / Données | License / Droit d'usage | Documentation |
|:--------------|:------------------------|:--------------|
| [OpenStreetMap](https://www.openstreetmap.org/) | [License Open Data Commons Open Database License (ODbL)](https://opendatacommons.org/licenses/odbl/) | [OpenStreetMap Wiki](https://wiki.openstreetmap.org/wiki/Main_Page) |

### Librairies Python

Voir le [README](https://github.com/thomleysens/Tutoriels_AME/tree/main#librairies-python-utilis%C3%A9es-dans-ce-d%C3%A9p%C3%B4t).

## Liens utiles

* [Librairie Python Panel](https://panel.holoviz.org/)
* [Convertir une application Panel pour la mise en ligne sur GitHub Pages](https://panel.holoviz.org/how_to/wasm/convert.html)
* [OpenStreetMap (*données cartographiques open source*)](https://www.openstreetmap.org/)
* [Construire un JupyterBook](https://jupyterbook.org/en/stable/start/your-first-book.html)